In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import mannwhitneyu, norm
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.stats.proportion import confint_proportions_2indep

from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.power import GofChisquarePower

from statsmodels.stats.power import TTestIndPower
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

import statsmodels.stats.proportion as prop
from device_detector import DeviceDetector

In [11]:
 
grouped_df_groups = ['paid_first', 'Paid_subsequent',  'Unpaid_first', "Unpaid_Subsequent"] 
all_results_list = []

In [12]:
def compare_ads_to_none_with_stats(df):
    """
    Compares each individual ad's middle vs. non-middle proportions to the 
    '(none)' ad group using a Chi-squared test.
    """
    #Isolate the reference group data: '(none)'
    df['is_middle'] = df['ad_position'] == 'Middle'
    df_none = df[df['cnt'] == '(none)']
    
    #NUmber of observations in the none group
    n_none = df_none.shape[0]
    
    # Calculate the proportion for the reference group 
    prop_none_middle = df_none['is_middle'].mean()
 
    # Get the counts for the '(none)' group
    none_middle_count = df_none['is_middle'].sum()
    none_non_middle_count = (~df_none['is_middle']).sum()

    #  Get the list of unique ads to compare (excluding '(none)')
    unique_ads = df['cnt'].unique()
    ads_to_compare = [ad for ad in unique_ads if ad != '(none)']

    # 3. Prepare the results list
    results = []

    #Loop through each individual ad and perform the comparison
    for ad_name in ads_to_compare:
        # Get data for the current ad
        df_ad = df[df['cnt'] == ad_name]
        
        # Calculate the proportion for the current ad (for display)
        prop_ad_middle = df_ad['is_middle'].mean()
        count_ad = df_ad.shape[0]

        # Get the counts for the current ad
        ad_middle_count = df_ad['is_middle'].sum()
        ad_non_middle_count = (~df_ad['is_middle']).sum()

        # if too little observations, skip
        if ad_middle_count < 5 or ad_non_middle_count < 5 or none_non_middle_count < 5 or none_middle_count < 5:
            results.append({
                'Ad Name': ad_name,
                'Ad Count': count_ad,              
                f'Prop. Middle (Ad)': f'{prop_ad_middle:.2%}',
                f'Prop. Middle ((none))': f'{prop_none_middle:.2%}',
                'Chi2 Statistic': f'{np.nan}',
                'P-Value': np.nan,
            'Cramér\'s V': f'{np.nan}',
            'Power': f'{np.nan}',
            'Total size': np.nan
            })
            continue
        
        # Build the 2x2 Contingency Table for the Chi-Squared Test
        # Rows: Position (Middle, Non-Middle)
        # Columns: Ad Group (Current Ad, (none))
        contingency_table = np.array([
            [ad_middle_count, none_middle_count],    
            [ad_non_middle_count, none_non_middle_count]  
        ])

        # Perform the Chi-squared test of independence
        # Returns: chi2 statistic, p-value, degrees of freedom, expected frequencies
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        
        # Calculate the Effect size  
        N = np.sum(contingency_table)
        rows, cols = contingency_table.shape
        k = min(rows, cols)
        v = np.sqrt(chi2 / (N * (k - 1)))
        pearson_residuals = (contingency_table - expected) / np.sqrt(expected)
        
        power_analysis = GofChisquarePower()

        # Calculate the power of the test 
        # And what should be total size, to have power = 0.80
        if v != 0:            
            power = power_analysis.solve_power(
                effect_size=v, 
                nobs=N, 
                n_bins=dof + 1, 
                alpha=0.05, 
                power=None 
            )
                        
            effect_size_h = prop.proportion_effectsize(prop_ad_middle, prop_none_middle )
            analysis = NormalIndPower()
            # In case oit fails to converge to a reuired n for ad group
            # set the required size to infinity. 
            try:
                with warnings.catch_warnings():
                    warnings.simplefilter("error", ConvergenceWarning)
                    
                    required_ratio = analysis.solve_power(
                        effect_size=effect_size_h,
                        nobs1=float(n_none),
                        ratio=None,
                        alpha=0.05,
                        power=0.80,
                        alternative='two-sided'
                    )
                    required_n_ad = np.ceil(required_ratio * n_none)

            except:
                # If the solver fails to converge OR any other math error happens
                required_n_ad = np.inf
            
        else:
            power = np.nan
            

        # Store the results
        results.append({
            'Ad Name': ad_name,
            'Ad Count': count_ad,           
            f'Prop. Middle (Ad)': f'{prop_ad_middle:.2%}',
            f'Prop. Middle ((none))': f'{prop_none_middle:.2%}',
            'Chi2 Statistic': f'{chi2:.2f}',
            'P. Res Ad middle': pearson_residuals[0][0],
            'P-Value': p_value,
            'Cramér\'s V': f'{v:.2f}',
            'Power': f'{power:.2f}',
            'Required N for Ads for 80% Power': required_n_ad, # 
            'Total size': N
        })
    comparison_df =  pd.DataFrame(results)
 
    if comparison_df.empty:
        return comparison_df
    
    return  comparison_df.sort_values(
        by='P-Value', 
        ascending=True, 
        na_position='last'
    ) 

     

In [13]:
def filter_by_mdm_list(df, paid_ads_list):
    """
    Splits a DataFrame into two based on whether the value in the 'mdm' column 
    is present in the provided list of paid ads. That is, the results are split in to group, where
    
    1. The ads are paid 
    2. The ads are free

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - paid_ads_list (list): The list of values (paid ad identifiers) to check against.
    
    Returns:
    - tuple: (df_in_list, df_not_in_list)
    """

    # Create the boolean mask for paid/unpaid status
    mask_in_list = df['mdm'].isin(paid_ads_list)

    # Filtered DataFrames based on ad status
    df_paid = df[mask_in_list].copy()
    df_unpaid = df[~mask_in_list].copy()
 
    COL = 'is_subsequent_purchase'

    # Here, we filter the already split df_paid by looking at the column within df_paid
    df_paid_first = df_paid[df_paid[COL] == 0].copy()
    df_paid_subsequent = df_paid[df_paid[COL] == 1].copy()

    # --- B. Unpaid Ad Split (The corrected section) ---
    # We filter the already split df_unpaid by looking at the column within df_unpaid
    df_unpaid_first = df_unpaid[df_unpaid[COL] == 0].copy()
    df_unpaid_subsequent = df_unpaid[df_unpaid[COL] == 1].copy()

    # Return the four resulting DataFrames in a clear order
    return (df_paid_first, df_paid_subsequent, 
            df_unpaid_first, df_unpaid_subsequent)

In [14]:
def hodges_lehmann_ci(x, y, alpha=0.05):
    # Convert Series to NumPy arrays to avoid Pandas NotImplementedError
    x_arr = x.values if hasattr(x, 'values') else np.asarray(x)
    y_arr = y.values if hasattr(y, 'values') else np.asarray(y)
    
    n1, n2 = len(x_arr), len(y_arr)
    
    #  Calculate all pairwise differences using numpy arrays
    diffs = np.sort(np.subtract.outer(x_arr, y_arr).ravel())
    
    # calulate mu and sigma
    mu_u = n1 * n2 / 2
    sigma_u = np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12)
    z = norm.ppf(1 - alpha / 2)
    # find k for interval values
    k = int(round(mu_u - z * sigma_u))

    k = max(1, k) 
    
    lower_ci = diffs[k - 1]
    upper_ci = diffs[len(diffs) - k]
    hl_estimate = np.median(diffs)
    
    return hl_estimate, lower_ci, upper_ci

In [15]:
 

def calculate_mann_whitney_metrics(group_ad_series, group_none_series, alpha=0.05, min_nr_samples = 5):
    """
    Performs a one-tailed Mann-Whitney U test (less alternative) 
    and calculates the effect size (r) and observed power.
    
    This tests, if group of ads has statistically less revenue, than group of ads '(none)'
    
    """
    
    N1 = len(group_ad_series)
    N2 = len(group_none_series)
    N_total = N1 + N2
    
    # If samples are too little, skip
    if N1 < min_nr_samples or N2 < min_nr_samples + 5:
        return {'U_Stat': np.nan, 
                'P_Value': np.nan, 
                'Z_Score': np.nan,
                'Effect_Size_r': np.nan,
                'Effect_Size_d_approx': np.nan,
                'Observed_Power': np.nan,
                'Mediann Difference': np.median(group_ad_series) - np.median(group_none_series),
                'None group Median': np.median(group_none_series),
                'required size': np.nan,
                'difference lower CI': np.nan,
                'difference upper CI': np.nan}
        
    # Mann-Whitney U Test (with Z-approximation)
    u_statistic, p_value = mannwhitneyu(
        x=group_ad_series, 
        y=group_none_series, 
        alternative='less', 
        method='auto',
        nan_policy='omit'
    )
     
    # in case P-value is too little and cause numeric problems:
    TINY_P_VALUE = 1e-28

    if p_value < TINY_P_VALUE:
        safe_p_value = TINY_P_VALUE
    elif p_value > 1 - TINY_P_VALUE:
        safe_p_value = 1 - TINY_P_VALUE
    else:
        safe_p_value = p_value
        
    # From given p-value, calculate the Z-statistic value 
    Z_score = norm.ppf(safe_p_value) 

    #  Use the Z statistic value, to calculate the effect size (rank correlation)
    effect_size_r_rb = Z_score / np.sqrt(N_total)
 
    r_clipped = np.clip(effect_size_r_rb, -0.99, 0.99)
    # For power calculation, Cohens d approximation was suggested:
    d_approx =  2 * r_clipped / np.sqrt(1 - r_clipped**2)
 
    # Calculates the power 
    power_analysis = TTestIndPower()

    # 4. Solve for power
    power_result = power_analysis.solve_power(
        effect_size = d_approx,
        nobs1 = N1,               
        alpha = alpha,
        ratio =  N2 / N1,
        alternative = 'smaller',
        power = None             
    )
    
    # Calculates the requied size, so that power would be = 0.80
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        
        
        # Run the function (it will return a number, maybe np.inf, maybe a huge number)
        # Fix the None group size, and find ration for power = 80% and then find required N for ad group
        ratio_required_per_group = power_analysis.solve_power(
            effect_size=d_approx,
            nobs1=float(N2),               
            alpha=float(alpha),
            ratio=None,
            alternative='smaller',
            power=0.80
        )
        N_required_per_group = N2 * ratio_required_per_group
    if isinstance(N_required_per_group, (np.ndarray, list)):
    # Assuming it's a single-element array/list, get the scalar value
        N_required_per_group =  np.inf
 
        
    if np.isfinite(d_approx):
        d_approx_rounded = round(d_approx, 3)
    else:
        d_approx_rounded = np.nan # If it truly is NaN/inf, keep it NaN
 
    # Calculate the confidence intervals for medians:
    
    # HL Estimate is under 'CLES' (Common Language Effect Size) 
    # but the CI for the location shift is also provided
    
    # Returns median, lower interval adn upper interval:
    confidence_intervals = hodges_lehmann_ci(group_ad_series, group_none_series, alpha=alpha)
 
    
    return {
        'U_Stat': u_statistic, 
        'P_Value': safe_p_value, 
        'Z_Score': Z_score,
        'Effect_Size_r': round(effect_size_r_rb, 3),
        'Effect_Size_d_approx': d_approx_rounded,
        'Observed_Power': round(power_result, 3),
        'Mediann Difference':   confidence_intervals[0],
        'None group Median': np.median(group_none_series),
        'required size': np.ceil(N_required_per_group),
        'difference lower CI': confidence_intervals[1],
        'difference upper CI': confidence_intervals[2]
    }

def run_all_ad_tests(df_revenue, none_group_name='(none)', revenue_col='atributed_revenue', ad_col = 'cnt'):
    """ Runs the test for all ads and includes effect size and power. """
    
    all_campaigns = df_revenue[ad_col].unique()
    results_list = []
    
    group_none_series = df_revenue[df_revenue[ad_col] == none_group_name][revenue_col].dropna()
    
    if group_none_series.empty:
        print(f"Error: The comparison group '{none_group_name}' is empty.")
        return pd.DataFrame()

    print(f"Testing {len(all_campaigns) - 1} campaigns against '{none_group_name}'...")
    
    for ad_name in all_campaigns:
        if ad_name == none_group_name:
            continue
        
        group_ad_series = df_revenue[df_revenue[ad_col] == ad_name][revenue_col].dropna()
        
        # Calculate N1 and N2 for inclusion in the summary
        N1 = len(group_ad_series)
        N2 = len(group_none_series)

        test_results = calculate_mann_whitney_metrics(group_ad_series, group_none_series, min_nr_samples=5)
        
        test_results['Ad_Campaign'] = ad_name
        test_results['N_Ad'] = N1
        test_results['N_None'] = N2
        results_list.append(test_results)
        
    df_results = pd.DataFrame(results_list)
    
    if df_results.empty:
        return df_results   
    
    df_results = df_results.sort_values(by='P_Value', ascending=True)
    return df_results

In [16]:
each_ad_revenue_df = pd.read_csv(
    'each_ad_revenue_df.csv',
    sep=',',
    engine='python',
    quotechar='"',
    encoding='utf-8',
    on_bad_lines='warn'
)

In [17]:
each_ad_revenue_df.head()

,mdm,cnt,ad_position,session_type,atributed_revenue,is_subsequent_purchase,Offer
0,unpaid,(none),First/Last,Single-Ad,65.806554,0,A_Offer
1,paid,A_cnt,First/Last,Single-Ad,64.935230,1,A_Offer
2,unpaid,(none),First,Multi-Ad,35.513416,0,B_Offer
3,paid,B_cnt,First,Multi-Ad,42.466672,0,B_Offer
4,paid,(none),First,Multi-Ad,20.063971,0,B_Offer


In [18]:
  
unique_country = 'PL'
# The Ads are filtered by - paid or free. 
grouped_df = filter_by_mdm_list(each_ad_revenue_df, ['paid'])

# Goes trough each group - free/paid and first/subsequent 
for df_tite, df in zip(grouped_df_groups, grouped_df):
    print("=====================================================")
    print(f" Results for {df_tite} ads:")
    print(f"\nTesting subset with {len(df)} rows.")
    
    if df.empty:
        print(f"One of the resulting DataFrames is empty for country group {unique_country}. Skipping tests.")
    else:
        # Makes a seperate Dir for each country, where results are stored as .csv 
        os.makedirs(f'{unique_country}_atskaite', exist_ok=True)
        
        # Test each a/d against the '(none)' group in terms of revenue
        test_results = run_all_ad_tests(df)
        print("Testing ad revenue against (none):")
        print(test_results.head())
    
        
        # Test each ad's proportion of middle vs. non-middle against '(none)'
        test_for_props = compare_ads_to_none_with_stats(df)
        print("Testing ad position proportions against (none):")
        print(test_for_props.head())
        file_path_csv = f'{unique_country}_atskaite/2_{unique_country}_{df_tite}_proportions.csv'
        test_for_props.to_csv(file_path_csv, index=False)

        file_path_csv = f'{unique_country}_atskaite/1_{unique_country}_{df_tite}_mann_whitney_results.csv'
        test_results.to_csv(file_path_csv, index=False)
        print(f"Results saved to {file_path_csv}")
        
        # Compare thee distributions, but w.r.t. the Campain, not single ad.
        rows_with_campain = df[
            (df['ad_position'] == 'Last') | (df['ad_position'] == 'First/Last')
        ].copy()
        test_results = run_all_ad_tests(rows_with_campain, ad_col='Offer')
        print("Testing ad revenue against (none) by Campaigns:")
        print(test_results.head())
        file_path_csv = f'{unique_country}_atskaite/5_{unique_country}_{df_tite}_mann_whitney_results_with_campains.csv'
        test_results.to_csv(file_path_csv, index=False)
        
        # Do ads in general perform better than non-ads? (groups)
        paid_first = df.copy()
        mask_is_ad = paid_first['cnt'] != '(none)'
        paid_first.loc[mask_is_ad, 'cnt'] = 'is_ad'

        # 3. Run the tests (assuming this returns a DataFrame)
        test_results = run_all_ad_tests(paid_first)
        print("Testing ad revenue against (none) by Ad vs. No Ad when combining all ads:")
        print(test_results.head())
        test_results['Category'] = df_tite
        # 4. Append the results to the list
        all_results_list.append(test_results)


# Is there a dffernece, if all ads are combinined in First/Sbusequint groupd and tested against (none)
subsequent_df = pd.concat([grouped_df[1], grouped_df[3]], axis=0)
first_df = pd.concat([grouped_df[0], grouped_df[2]], axis=0)

file_path_csv_sub = f'{unique_country}_atskaite/3_subsequent_freeAndpayed_mann_whitney.csv'
file_path_csv_first = f'{unique_country}_atskaite/3_first_freeAndpayed_mann_whitney.csv'

if not subsequent_df.empty:
    print("Results for Subsequent Purchase Ads:")
    test_results = run_all_ad_tests(subsequent_df)
    # print(test_results.head(20))
    test_results.to_csv(file_path_csv_sub, index=False)
    print(f"Results saved to {file_path_csv_sub}")

if not first_df.empty:
    file_path_csv = f'first_purchase_mann_whitney_results.csv'
    print("Results for FirstPurchase Ads:")
    test_results = run_all_ad_tests(first_df)
    print("First Purchase Ads Results:")
    print(test_results.head())
    test_results.to_csv(file_path_csv_first, index=False)
    print(f"Results saved to {file_path_csv_first}")
else:
    print("First Purchase Ads DataFrame is empty. Skipping test.")

path_for_comined_ads = f'{unique_country}_atskaite/4_combined_mann_whitney.csv'
final_results_df = pd.concat(all_results_list, ignore_index=True)
final_results_df.to_csv(path_for_comined_ads, index=False)
print(final_results_df.head(4))    

 Results for paid_first ads:

Testing subset with 1736 rows.
Testing 181 campaigns against '(none)'...
Testing ad revenue against (none):
     U_Stat       P_Value   Z_Score  Effect_Size_r  Effect_Size_d_approx  \
46   1795.0  7.816685e-16 -7.971836         -0.239                -0.493   
11   7128.0  8.596306e-06 -4.298541         -0.129                -0.261   
8     806.0  3.366750e-03 -2.709745         -0.082                -0.165   
20   6947.0  7.759876e-03 -2.420018         -0.073                -0.146   
2   13160.0  1.050495e-02 -2.307807         -0.069                -0.139   

    Observed_Power  Mediann Difference  None group Median  required size  \
46           0.824          -30.348429          39.743565           27.0   
11           0.370          -21.614455          39.743565          100.0   
8            0.101          -28.331296          39.743565          288.0   
20           0.156          -13.426781          39.743565          395.0   
2            0.192       